# Self join

## Edinburgh Buses
[Details of the database](https://sqlzoo.net/wiki/Edinburgh_Buses.) Looking at the data

```
stops(id, name)
route(num, company, pos, stop)
```

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@localhost/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 100)

 ····


## 1.
How many **stops** are in the database.

In [2]:
stops = pd.read_sql_table('stops', engine)
route = pd.read_sql_table('route', engine)

In [6]:
pd.DataFrame(stops[['id']].count())

,0
id,246


## 2.
Find the **id** value for the stop 'Craiglockhart'

In [7]:
stops.loc[stops['name']=='Craiglockhart', ['id']]

,id
52,53


## 3.
Give the **id** and the **name** for the **stops** on the '4' 'LRT' service.

In [9]:
a = stops.merge(route, how='left', left_on='id', right_on='stop')
a.loc[(a['num']=='4') & (a['company']=='LRT'), ['id', 'name']]

,id,name
45,19,Bingham
263,53,Craiglockhart
397,85,Fairmilehead
541,115,Haymarket
566,117,Hillend
692,149,London Road
834,177,Northfield
846,179,Oxgangs
928,194,Princes Street


## 4. Routes and stops

The query shown gives the number of routes that visit either London Road (149) or Craiglockhart (53). Run the query and notice the two services that link these stops have a count of 2. Add a HAVING clause to restrict the output to these two routes.

In [14]:
(route.loc[route['stop'].isin((149, 53)),
         ['company', 'num', 'stop']]
    .groupby(['company', 'num'])
    .count()
    .reset_index()
    .query('stop==2'))

,company,num,stop
7,LRT,4,2
11,LRT,45,2


## 5.
Execute the self join shown and observe that b.stop gives all the places you can get to from Craiglockhart, without changing routes. Change the query so that it shows the services from Craiglockhart to London Road.

In [17]:
(route.merge(route, how='inner', 
             left_on=['company', 'num'], right_on=['company', 'num'])
    .query('(stop_x==53) & (stop_y==149)')
    .loc[:, ['company', 'num', 'stop_x', 'stop_y']])

,company,num,stop_x,stop_y
3018,LRT,4,53,149
8680,LRT,45,53,149


## 6.
The query shown is similar to the previous one, however by joining two copies of the **stops** table we can refer to **stops** by **name** rather than by number. Change the query so that the services between 'Craiglockhart' and 'London Road' are shown. If you are tired of these places try 'Fairmilehead' against 'Tollcross'

In [19]:
(route.merge(route, how='inner', 
            left_on=['company', 'num'], right_on=['company', 'num'])
    .merge(stops, left_on='stop_x', right_on='id')
    .merge(stops, left_on='stop_y', right_on='id')
    .query('(name_x=="Craiglockhart") & (name_y=="London Road")')
    .loc[:, ['company', 'num', 'name_x', 'name_y']])

,company,num,name_x,name_y
361,LRT,4,Craiglockhart,London Road
362,LRT,45,Craiglockhart,London Road


## 7. [Using a self join](https://sqlzoo.net/wiki/Using_a_self_join)

Give a list of all the services which connect stops 115 and 137 ('Haymarket' and 'Leith')

In [20]:
(route.merge(route, how='inner', 
            left_on=['company', 'num'], right_on=['company', 'num'])
    .query('stop_x==115 & stop_y==137')
    .loc[:, ['company', 'num']]
    .drop_duplicates())

,company,num
6643,LRT,12
6941,LRT,2
7256,LRT,2A
10944,SMT,C5
11192,LRT,22
11321,LRT,25


## 8.
Give a list of the services which connect the stops 'Craiglockhart' and 'Tollcross'

In [21]:
(route.merge(route, how='inner', 
            left_on=['company', 'num'], right_on=['company', 'num'])
    .merge(stops, left_on='stop_x', right_on='id')
    .merge(stops, left_on='stop_y', right_on='id')
    .query('name_x=="Craiglockhart" & name_y=="Tollcross"')
    .loc[:, ['company', 'num']])

,company,num
8325,LRT,47
8326,LRT,45
8327,LRT,27
8328,LRT,10


## 9.
Give a distinct list of the **stops** which may be reached from 'Craiglockhart' by taking one bus, including 'Craiglockhart' itself, offered by the LRT company. Include the company and bus no. of the relevant services.

In [24]:
(route.merge(route, how='inner', 
            left_on=['company', 'num'], right_on=['company', 'num'])
    .merge(stops, left_on='stop_x', right_on='id')
    .merge(stops, left_on='stop_y', right_on='id')
    .query('name_x=="Craiglockhart" & company=="LRT"')
    .loc[:, ['name_y', 'company', 'num']]
    .drop_duplicates())

,name_y,company,num
361,London Road,LRT,4
362,London Road,LRT,45
1395,Hanover Street,LRT,47
1396,Hanover Street,LRT,45
1397,Hanover Street,LRT,27
1812,Canonmills,LRT,47
1813,Canonmills,LRT,27
2074,Crewe Toll,LRT,27
3641,Leith,LRT,10
5130,Princes Street,LRT,4


## 10.
Find the routes involving two buses that can go from **Craiglockhart** to **Lochend**.
Show the bus no. and company for the first bus, the name of the stop for the transfer,
and the bus no. and company for the second bus.

> _Hint_    
> Self-join twice to find buses that visit Craiglockhart and Lochend, then join those on matching stops.

In [30]:
bus1 = (route.merge(route, how='inner', 
            left_on=['company', 'num'], right_on=['company', 'num'])
    .merge(stops, left_on='stop_x', right_on='id')
    .merge(stops, left_on='stop_y', right_on='id')
    .query('name_x=="Craiglockhart"')
    .loc[:, ['name_y', 'company', 'num', 'stop_y']]
    .drop_duplicates())
bus2 = (route.merge(route, how='inner', 
            left_on=['company', 'num'], right_on=['company', 'num'])
    .merge(stops, left_on='stop_x', right_on='id')
    .merge(stops, left_on='stop_y', right_on='id')
    .query('name_y=="Lochend"')
    .loc[:, ['stop_x', 'company', 'num']]
    .drop_duplicates())
(bus1.merge(bus2, left_on='stop_y', right_on='stop_x')
    .loc[:, ['num_x', 'company_x', 'name_y', 'num_y', 'company_y']])

,num_x,company_x,name_y,num_y,company_y
0,4,LRT,London Road,87,LRT
1,4,LRT,London Road,34,LRT
2,4,LRT,London Road,35,LRT
3,4,LRT,London Road,87A,LRT
4,4,LRT,London Road,42,LRT
5,4,LRT,London Road,46A,LRT
6,4,LRT,London Road,20,LRT
7,4,LRT,London Road,C5,SMT
8,4,LRT,London Road,65,LRT
9,45,LRT,London Road,87,LRT
